In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/조인설

/content/drive/MyDrive/조인설


In [3]:
! python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 31.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [4]:
! pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import copy
import numpy as np
import pandas as pd
import cv2
from glob import glob
import os
import os.path
import argparse
import json
import matplotlib.pyplot as plt
import argparse
import subprocess
import sys
from pathlib import Path
from typing import NamedTuple
import ffmpeg

# openpose setup
from src import model
from src import util
from src.body import Body
from src.hand import Hand

modelpt='model'
body_estimation = Body(modelpt+'/body_pose_model.pth')
hand_estimation = Hand(modelpt+'/hand_pose_model.pth')

### Keypoint Estimation

In [6]:
video_file = 'Test Video/NIA_SL_WORD1128_REAL18_F.mp4'

In [7]:
class OpenPose():
    def __init__(self,modelpath):  
        self.body_estimation = Body(modelpath+'/body_pose_model.pth')
        self.hand_estimation = Hand(modelpath+'/hand_pose_model.pth')

    def handpt(self, oriImg):    
        candidate, subset = self.body_estimation(oriImg)
        canvas = copy.deepcopy(oriImg)
        canvas = util.draw_bodypose(canvas, candidate, subset)

        hands_list = util.handDetect(candidate, subset, oriImg)
        all_hand_peaks = []
        cnt=0
        
        for x, y, w, is_left in hands_list:
            peaks = self.hand_estimation(oriImg[y:y+w, x:x+w, :])
            peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], peaks[:, 0]+x)
            peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y)

            all_hand_peaks.append(peaks)

        return all_hand_peaks

In [8]:
class FFProbeResult(NamedTuple):
    return_code: int
    json: str
    error: str

def ffprobe(file_path) -> FFProbeResult:
    command_array = ["ffprobe",
                     "-v", "quiet",
                     "-print_format", "json",
                     "-show_format",
                     "-show_streams",
                     file_path]
    result = subprocess.run(command_array, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    return FFProbeResult(return_code=result.returncode,
                         json=result.stdout,
                         error=result.stderr)

def process_frame(frame, body=True, hands=True):
    canvas = copy.deepcopy(frame)
    if body:
        candidate, subset = body_estimation(frame)
        canvas = util.draw_bodypose(canvas, candidate, subset)
    if hands:
        hands_list = util.handDetect(candidate, subset, frame)
        all_hand_peaks = []
        for x, y, w, is_left in hands_list:
            peaks = hand_estimation(frame[y:y+w, x:x+w, :])
            peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], peaks[:, 0]+x)
            peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y)
            all_hand_peaks.append(peaks)
        canvas = util.draw_handpose(canvas, all_hand_peaks)
    return canvas

class Writer():
    def __init__(self, output_file, input_fps, input_framesize, input_pix_fmt,
                 input_vcodec):
        if os.path.exists(output_file):
            os.remove(output_file)
        self.ff_proc = (
            ffmpeg
            .input('pipe:',
                   format='rawvideo',
                   pix_fmt="bgr24",
                   s='%sx%s'%(input_framesize[1],input_framesize[0]),
                   r=input_fps)
            .output(output_file, pix_fmt=input_pix_fmt, vcodec=input_vcodec)
            .overwrite_output()
            .run_async(pipe_stdin=True)
        )

    def __call__(self, frame):
        self.ff_proc.stdin.write(frame.tobytes())

    def close(self):
        self.ff_proc.stdin.close()
        self.ff_proc.wait()

In [9]:
if __name__ == "__main__":
    flist=list()
    pose = OpenPose(modelpt)
    arr = []
    temp_peaks = [] #이전 frame의 keypoint
    
    vidcap = cv2.VideoCapture(video_file)

    # get video file info
    ffprobe_result = ffprobe(video_file)
    info = json.loads(ffprobe_result.json)
    videoinfo = [i for i in info["streams"] if i["codec_type"] == "video"][0]
    input_fps = videoinfo["avg_frame_rate"]
    # input_fps = float(input_fps[0])/float(input_fps[1])
    input_pix_fmt = videoinfo["pix_fmt"]
    input_vcodec = videoinfo["codec_name"]
    
    # define a writer object to write to a movidified file
    postfix = info["format"]["format_name"].split(",")[0]
    output_file = ".".join(video_file.split(".")[:-1])+".processed." + postfix

    count = 0
    writer = None

    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if not ret:
          break

        # 영상 길이에 따라 이미지 추출 간격 조정
        total_frame_count = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
        image = int(total_frame_count/20)

        # frame 당 하나씩 이미지 추출
        if(int(vidcap.get(1)) % image == 0 and count < 20):
          position = pose.handpt(frame)
          out_arr = np.ravel(position, order='C') # shape 수정, 1차원으로 만들기(0:84)

          if out_arr.size == 44:
            out_arr = temp_peaks.reshape(1, 84)
          if out_arr.size == 84:
            temp_peaks = out_arr
            out_arr = out_arr.reshape(1, 84)

          arr = np.append(arr, out_arr) # 뒤에 append
          
          posed_frame = process_frame(frame)

          if writer is None:
            input_framesize = posed_frame.shape[:2]
            writer = Writer(output_file, input_fps, input_framesize, input_pix_fmt,
                        input_vcodec)
          # write the frame
          writer(posed_frame)
          count += 1

    vidcap.release()
    writer.close()
    cv2.destroyAllWindows()

### Prediction

In [10]:
from sklearn.preprocessing import Normalizer
import tensorflow as tf

X = arr.reshape(1,1680)
transformer = Normalizer().fit(X)
X = transformer.transform(X)

model = tf.keras.models.load_model('class 100/model_100.h5')
y_predict = model.predict(X)
label = y_predict[0].argmax()

CLASSES = ["춥다", "덥다", "먹구름", "바람", "비", "온도", "장마", "햇빛", "싫다", "좋다", "양산", "체온", "하늘", "강풍", "따뜻하다", "홍수", "예정", "사계절", "눈", "뙤약볕",
           "감기", "에어컨", "체감", "경고", "찻길", "출근", "강", "파도", "어젯밤", "방금", "어떻게", "자정", "아까", "최대", "최소", "평일", "깜깜하다", "틀림없다", "매일매일", "혹시",
           "들어맞다", "사라지다", "오래도록", "찰나", "이미", "순식간", "모르다", "불가능", "충분", "불안","당황", "원하다", "무섭다", "빠르다", "즐겁다", "마지막", "오다", "깨끗하다", "행복", "떨다",
           "조심","변덕","망설이다","희망","힘들다","화나다","놀라다","사람","우울","빨리", "못하다","시원하다","기억","무지개","불신","대략","피곤","시작","충격","조용하다",
           "상상","예견","난감하다","알려주다","잊어버리다","깜빡하다","결코","제법","뜻밖","격노","짜증내다","강물","달","뜨겁다","별","호수","일몰","일출","적중하다","정말"]

print('Label: ', label)
print('Translation: ', CLASSES[label])

1/1 [==============================] - 2s 2s/step
Label:  50
Translation:  당황


### Recognition Result

In [11]:
import cv2
from IPython.display import HTML, display, Javascript
from base64 import b64encode

def show_video(video_path, class_name, video_width = 600):
  
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video><div style="position:absolute; top:55px; left:20px; font-size:20px; color:#ffffff;">{class_name}</div>""")

print('Label: ', label)
print('Translation: ', CLASSES[label])
show_video(output_file, CLASSES[label])

Label:  50
Translation:  당황
